## 数据背景

    当前社会，几乎人人都离不开手机。人们每天使用手机进行上网、购物、社交、办公等等。手机几乎承载了一个人全部的行为和偏好。那么我们对手机设备的用户画像就变得越来越重要了，它不但能帮助APP企业了解自己用户的行为特征，迭代产品；还能帮助企业更精准的投放互联网广告，节省广告费用。通过设备数据、应用数据等，进行模型训练，并用训练好的模型预测用户的性别和年龄。

## 数据介绍
    标签数据: 一个设备ID会有唯一的性别和年龄段。性别有1、2两种可能值，分别代表男和女。年龄段有0到10十一种可能，分别代表不同的年龄段，且数值越大相应的年龄越大。一个设备只属于一个唯一的类别（性别+年龄段），共有22个类别(一个22分类问题)。
    
   - 设备数据：每个设备上的应用安装列表，设备应用名都进行了hash处理
   - 应用数据：每个设备上各个应用的打开、关闭行为数据
   - 机型数据：每个设备的品牌和型号
   - APP数据：每个应用的类别信息

## 复杂特征工程

- 俗话说:数据和特征决定了机器学习的上限，而模型和算法只是不断地逼近这个上限。

### 特征工程:

 - 设备数据 : 应用bag of word模型,把设备上应用安装列表转换为向量
     - device_data
 - 应用数据 : 挖掘用户的偏好,行为,和时间特征
     - h1_data, h2_data, h3_data
 - 机型数据 : 数据清洗，去掉冗余
     - model_data
 - APP数据 : 处理缺失的类别信息
     - app_data
 - 标签数据 : 性别和年龄
     - gender_age_label(可拆分为：gender_label, age_label)
 - 全部数据 : (设备+应用+机型+APP）
 　　- all_data

## 建模

### 模型的结构
    模型是一个两级的Stacking model,第一级模型，根据提取的特征建模，主要用到模型是MLP和GBDT,二级模型，把一级模型的输出作为二级模型的输入，训练二级模型(MLP)．

In [4]:
import numpy as np
import xgboost as xgb
from sklearn.neural_network import MLPClassifier

In [2]:
def training_code(train_x, test_x, labels, model, n_folds=5):
    '''
    train_x/test_x : 特征数据
    label  : 标签
    model  : model(MLP, GBDT)
    n_folds: k-fold(k折交叉的折数)
    '''
    test_pred = np.zeros(test_x.shape[0])
    oof = np.zeros(train_x.shape[0])
    SKF = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=2019)
    for i, (train_index, valid_index) in enumerate(SKF.split(train_df, labels)):
        x_train = train_x.iloc[train_index, :]
        y_train = labels[train_index]
        x_valid = train_x.iloc[valid_index, :]
        y_valid = labels[valid_index]
        model.fit(x_train, y_train)
        test_pred += model.predict(test_df)/n_folds
        oof[valid_index] = model.predict_proba(x_valid)
        gc.enable()
        del x_train, x_valid, y_train, y_valid
        gc.collect()
    return oof, test_pred

In [5]:
mlp = MLPClassifier(hidden_layer_sizes=(300,300,300), activation='relu', solver='adam',
                    alpha=0.0001, batch_size=128, learning_rate='adaptive', 
                    learning_rate_init=0.00054321, power_t=0.5, max_iter=200,
                    shuffle=True, random_state=666, tol=0.005, verbose=False,
                    warm_start=False, momentum=0.9, early_stopping=True,
                    validation_fraction=0.1)

gbdt_1 = xgb.XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=1000,
                       objective='binary:logistic',booster='gbtree',
                       n_jobs=-1, gamma=0, min_child_weight=1, max_delta_step=0,
                       subsample=1, colsample_bytree=0.9, colsample_bylevel=0.9,
                       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5,
                       random_state=666)

gbdt_2 = xgb.XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=1000,
                           objective='multi:softprob', booster='gbtree', n_jobs=-1,
                           subsample=1, colsample_bytree=0.6, colsample_bylevel=1.,
                           reg_alpha=0, reg_lambda=0.5, scale_pos_weight=1,
                           base_score=0.5, max_delta_step = 0, random_state=666)

In [ ]:
# 22 分类
train_set_1 = [device_train, h1_train, h2_train, h3_train, model_train, app_train, all_train]
test_set_1 = [device_test, h1_test, h2_test, h3_test, model_test, app_test, all_test]
# 2/11分类
train_set_2 = [h3_train, all_train]
test_set_2 = [h3_test, all_test]

In [6]:
def stacking(train_set, test_set, labels, model, n_folds=5):
    OOF = []
    Test_pred = []
    for train_x, test_x in zip(train_set, test_set):
        oof, test_pred = training_code(train_x, test_x, labels, model, n_folds)
        OOF.append(oof)
        Test_pred.append(test_pred)
    stack_train = np.concatenate(OOF, axis=1)
    stack_test = np.concatenate(Test_pred, axis=1)
    return stack_train, stack_test

In [ ]:
# 22分类
stack_train_1, stack_test_1 = stacking(train_set_1, test_set_1, gender_age_label, mlp, 5)
stack_train_2, stack_test_2 = stacking(train_set_1, test_set_1, gender_age_label, gbdt_2, 5)
# 2 分类
stack_train_3, stack_test_3 = stacking(train_set_2, test_set_2, gender_label, gbdt_1, 5)
# 11分类
stack_train_4, stack_test_4 = stacking(train_set_2, test_set_2, age_label, gbdt_2, 5)

In [ ]:
# concatenate
stack_train = np.concatenate([stack_train_1,stack_train_2,stack_train_3,stack_train_4],axis=1)
stack_test = np.concatenate([stack_test_1,stack_test_2,stack_test_3,stack_test_4],axis=1)

In [ ]:
mlp.fit(stack_train, gender_age_label)
result = mlp.predict(stack_test)